In [1]:
import numpy as np
import pandas as pd

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [13]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [21]:
df['fever'].sample(5)

82     98.0
4     101.0
31    103.0
41      NaN
75    102.0
Name: fever, dtype: float64

In [39]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

## TTS

In [14]:
x = df.drop(columns='has_covid', axis=1)
y = df['has_covid']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

(80, 5)

# Normal

In [23]:
# filling the missing vals in fever
si = SimpleImputer(strategy='mean', missing_values=np.nan)
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.transform(x_test[['fever']])

In [25]:
x_train_fever.shape

(80, 1)

In [26]:
# applying ordinal on cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])

In [46]:
x_train_cough.shape

(80, 1)

In [40]:
# gender and city - ode 
ode = OneHotEncoder(sparse_output=False, drop='first')
x_train_gender_city = ode.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ode.transform(x_test[['gender', 'city']])

In [41]:
x_train_gender_city.shape

(80, 4)

In [58]:
# storing age

# x_train_age = x_train['age']
# x_test_age = x_test['age']

x_train_age = x_train.drop(columns=['gender', 'city', 'fever', 'cough']).values
x_test_age = x_test.drop(columns=['gender', 'city', 'fever', 'cough']).values

In [59]:
x_train_age.shape   # it is 1D if it is (80, )

(80, 1)

In [77]:
# combining all
x_train_combined = np.concatenate((x_train_age, x_train_cough, x_train_fever, x_train_gender_city), axis=1)
x_test_combined = np.concatenate((x_test_age, x_test_cough, x_test_fever, x_test_gender_city), axis=1)

In [78]:
x_train_combined.shape

(80, 7)

In [82]:
x_test_combined

array([[ 65.        ,   0.        ,  99.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 13.        ,   1.        , 100.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 51.        ,   0.        , 104.        ,   1.        ,
          0.        ,   1.        ,   0.        ],
       [ 34.        ,   1.        ,  98.        ,   1.        ,
          0.        ,   1.        ,   0.        ],
       [ 69.        ,   1.        ,  98.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 47.        ,   1.        , 100.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 20.        ,   1.        , 102.        ,   1.        ,
          1.        ,   0.        ,   0.        ],
       [ 42.        ,   0.        , 101.        ,   1.        ,
          1.        ,   0.        ,   0.        ],
       [ 27.        ,   0.        , 100.        ,   1.        ,
          1.    

# With Steroids

In [68]:
from sklearn.compose import ColumnTransformer
tf = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [69]:
x_train_new = tf.fit_transform(x_train)

In [75]:
x_train_new.shape

(80, 7)

In [73]:
x_test_new = tf.transform(x_test)

In [83]:
x_test_new.shape

(20, 7)

In [81]:
x_test_new

array([[ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  65.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  13.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  51.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [102.        ,   1.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.    